In [1]:
from flask import Flask, request, send_file, jsonify
import base64
from PIL import Image
import io
import numpy as np
import cv2
import tensorflow as tf
app = Flask(__name__)
# Edge detection function using Canny edge detection
model = tf.keras.applications.MobileNetV2(weights='imagenet')
def detect_edges(img):
    edges = cv2.Canny(img, 100, 200)
    return edges

# Feature detection and extraction function using ORB algorithm
def detect_features(img):
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(img, None)
    img_with_keypoints = cv2.drawKeypoints(img, keypoints, None)
    return img_with_keypoints

def process_image(image):
    # Add your image processing functions here
    # For example, you could convert the image to grayscale and invert the colors
    image = Image.fromarray(image).convert('L')
    image = np.array(image)
    image = 255 - image
    return image
@app.route('/upload', methods=['POST'])
def upload_image():
    # receive the base64-encoded image from the client
    image_data = request.json['image']
    
    # convert the image back to a file
    with open('image.jpg', 'wb') as f:
        f.write(base64.b64decode(image_data))
    
    #save image and return image in response 
    

    return send_file('image.jpg', mimetype='image/jpg')
@app.route('/process-image1', methods=['POST'])
def process_image_route():
    image_file = request.files.get('image')
    image = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_COLOR)

    # Process the image
    image1 = process_image(image)
    image1 = detect_edges(image1)
    
    # Preprocess the image for object detection
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.resize(image, (224, 224))
    image = tf.expand_dims(image, 0)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)

    # Predict objects in the image
    predictions = model.predict(image)
    decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=5)[0]

    # Extract the top predicted label and confidence
    top_label = decoded_predictions[0][1]
    top_confidence = decoded_predictions[0][2]

    # Convert the segmented image to a numpy array
    ret, jpeg = cv2.imencode('.jpg', image1)
    encoded_image = jpeg.tobytes()

    # Create a response JSON object
    response = {
        'label': top_label,
        'confidence': float(top_confidence),
        'image': base64.b64encode(encoded_image).decode('utf-8')
    }

    # Return the response as JSON
    return jsonify(response)

    # Return the encoded image as a response
    # return Response(encoded_image, mimetype='image/jpeg')
if __name__ == '__main__':
    #run app in debug mode on port 5000
    app.run(host='0.0.0.0', port=5000)
    


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.7:5000
Press CTRL+C to quit


1/1 [==============================] - 1s 723ms/step


127.0.0.1 - - [13/May/2023 19:30:25] "POST /process-image1 HTTP/1.1" 200 -


1/1 [==============================] - 0s 40ms/step


...

192.168.1.5 - - [13/May/2023 19:46:24] "POST /process-image1 HTTP/1.1" 200 -


1/1 [==============================] - 0s 41ms/step


192.168.1.5 - - [13/May/2023 19:46:57] "POST /process-image1 HTTP/1.1" 200 -


1/1 [==============================] - 0s 40ms/step


192.168.1.5 - - [13/May/2023 19:48:32] "POST /process-image1 HTTP/1.1" 200 -


In [2]:
import cv2

# Edge detection function using Canny edge detection
def detect_edges(img):
    edges = cv2.Canny(img, 100, 200)
    return edges

# Feature detection and extraction function using ORB algorithm
def detect_features(img):
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(img, None)
    img_with_keypoints = cv2.drawKeypoints(img, keypoints, None)
    return img_with_keypoints

# Load the image
img = cv2.imread('image.jpg')

# Detect edges in the image
edges = detect_edges(img)

# Detect features in the image
img_with_keypoints = detect_features(img)

# Save the edited images
cv2.imshow('edges.jpg', edges)
cv2.imshow('features.jpg', img_with_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
